In [2]:
from collections import Counter
from tqdm import tqdm 
from colorama import Style,Fore
import os
from utils1 import openJson,saveJson
from dotenv import load_dotenv
load_dotenv()

True

# With Embedding model

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("dangvantuan/sentence-camembert-base")

In [4]:
from sentence_transformers import util

def calculateSimilarity(videoMD, documents):
    embedding_video = model.encode([videoMD], convert_to_tensor=True, normalize_embeddings=True)
    embeddings_documents = model.encode(documents, convert_to_tensor=True, normalize_embeddings=True)
    scores = util.cos_sim(embedding_video, embeddings_documents)[0]
    simScore = scores.mean().item()
    return simScore


In [9]:
Vocabulary = openJson("../jsons/Voc.json")

def construire_groupes(mots_cles, taille=25):
    phrases = []
    for i in range(0, len(mots_cles), taille):
        groupe = mots_cles[i:i+taille]
        phrase = ' '.join(groupe)
        phrases.append(phrase)
    return phrases

documents = construire_groupes(Vocabulary)

In [10]:
print(len(documents))
print(documents[7])

10
cultiver des légumineuses sustainable farming phytoépuration puits canadien greffage eaux grises caravane serre oignon perpétuel faux yourte batterie lithium pompe capteur solaire filtre doulton maison terre-paille culture lasagnes puits artésien fermentation jardinage pommes de terre pénurie alimentaire poulailler cultures biologiques agroecologie


## Test

In [11]:
requete = """
On a testé la cuisinière à bois... Vous connaissez?
🙋‍♂️Et vous, vous êtes-vous déjà servi d'une cuisinière à bois? Avez-vous des conseils à nous donner pour garantir son utilisation et la sécurité inhérente à cet appareil ancestral? ❔\n\n➕La cuisinière à bois, c'était clairement un bonus qu'on n'avait mis dans nos préférences en visitant la maison. On connaissait certains de ses avantages, mais en l'utilisant pour la première fois, non seulement on a été conquis, mais cette expérience a révélé de nombreux points positifs qu'on ignorait. On vous raconte tout ça dans cette nouvelle vidéo! \n\n🙅‍♂️J'ai oublié d'évoquer un des avantages dans cette vidéo: PRENDRE LE TEMPS! C'est tellement bon de prendre plus de temps pour faire sensiblement la même chose!\n\n✍️On a hâte d'avoir vos retours. Depuis cette vidéo, on est retournés une fois à nos fourneaux pour fêter la Thanksgiving canadienne, et on a définitivement adopté ce nouveau compagnon de route! En plus de tous les avantages cités dans cette vidéo, c'est ultra convivial!\n\n😎 Si c'est votre première fois, bienvenue sur notre chaîne. Ici on va parler autonomie bien sûr, mais transition en général, que ce soit vers des nouveaux modes de consommation, vers d'autres manières de voyager, vers un vivre-ensemble qui a du sens. En famille ou sans, c'est pareil, on est dans le même bateau.\n\nBonne vidéo, merci de nous soutenir en vous abonnant et à très vite dans les commentaires!\n\n00:00 On n'y connaissait rien\n01:12 On présente le four (avant utilisation)\n02:55 Premier essai: Pain...\n05:37 ... et plein d'autres trucs (on optimise)\n06:54 On a adoré cette première expérience!\n07:50 On déguste. Vous venez?\n\n( ͡❛ ᴗ ͡❛)✊ LA FAMILLE EN TRANSITION\n\n❤ Abonne-toi à la Chaîne pour ne rater aucune vidéo https://www.youtube.com/channel/UC2M77RNyW6O9JQhpj1-6GOA\n✌ Laisse-nous un pouce bleu si la vidéo t'a plue\n\nContacte-nous à lafamilleentransition@gmail.com",
 "#lafamilleentransition",
      "#LFET",
      "#Florian Guéret",
      "#Cuisinière à bois",
      "#comment cuisiner avec une cuisinière à bois",
      "#bois de chauffage",
      "#cuisiner comme au bon vieux temps",
      "#cuisiner à l'ancienne",
      "#cuisine",
      "#cuisiner au feu de bois",
      "#cuisine moderne",
      "#cuisine traditionnelle",
      "#transition",
      "#entransition",
      "#la famille",
      "#vlog famille",
      "#vlog",
      "#bretagne",
      "#pontivy",
      "#à l'abri de la pluie",
      "#comment cuisiner du pain",
      "#comment faire son pain en cocotte",
      "#faire son pain en baguette"
"""


calculateSimilarity(requete,documents)

0.05205638334155083

## Echantillon

In [12]:
videosF2 = openJson("../jsons/videosF2.json")
len(videosF2)

4504

In [13]:
import random

random.seed(42)

echantillon = random.sample(videosF2, 400)
saveJson("../jsons/echantillon.json",echantillon)


 json saved


## Run and Results 

In [3]:
echantillon = openJson("../jsons/echantillon.json")
len(echantillon)

400

In [15]:
approach_1_Results = []
for video in tqdm(echantillon):
    context = video['titre_video'] + video['description'] + ' '.join(video['tags'])
    score = calculateSimilarity(context,documents)
    #print(score,'  ',video['id_video'])
    video.update({'similarity':score})
    approach_1_Results.append(video)

saveJson("../jsons/approach_1_Results.json",approach_1_Results)

100%|██████████| 400/400 [07:48<00:00,  1.17s/it]


 json saved


- Analyzing

In [19]:
approach_1_Results = openJson("../jsons/approach_1_Results.json")
len(approach_1_Results)

400

In [20]:
sum_similarities = 0 
for item in approach_1_Results :
    sum_similarities+=item['similarity']
    
sum_similarities/len(approach_1_Results)

0.2867241595336236

In [21]:
similarites = [item["similarity"] for item in approach_1_Results]

nombre_total = len(similarites)
nombre_sup_015 = sum(s > 0.15 for s in similarites)
pourcentage_sup_015 = (nombre_sup_015 / nombre_total) * 100

print(f"Nombre de vidéos avec similarité > 0.15 : {nombre_sup_015}")
print(f"Pourcentage : {pourcentage_sup_015:.2f}%")


Nombre de vidéos avec similarité > 0.15 : 361
Pourcentage : 90.25%


In [22]:
videos_sous_015 = [item for item in approach_1_Results if item["similarity"] <= 0.15]
videos_sous_015 = sorted(videos_sous_015, key=lambda x: x["similarity"])

videos_sur_015 = [item for item in approach_1_Results if item["similarity"] > 0.15]
videos_sur_015 = sorted(videos_sur_015, key=lambda x: x["similarity"])

In [25]:
for vi_sous_015 in videos_sous_015:
    print(f"ID vidéo: {vi_sous_015['id_video']} - Similarité: {vi_sous_015['similarity']:.2f}")

ID vidéo: JRAWlxIlA7A - Similarité: -0.01
ID vidéo: DdzpreePx6Q - Similarité: 0.01
ID vidéo: Rjv7GeThofI - Similarité: 0.01
ID vidéo: wL_7d-gH7ek - Similarité: 0.02
ID vidéo: lSoWicUMVlM - Similarité: 0.05
ID vidéo: ZQRZviGuF4g - Similarité: 0.05
ID vidéo: SrFPFZ14loQ - Similarité: 0.06
ID vidéo: uIib2jT2r3w - Similarité: 0.07
ID vidéo: 6H1_LUx-rBc - Similarité: 0.07
ID vidéo: JG4XtGYcwtg - Similarité: 0.07
ID vidéo: SGslcr5UEwQ - Similarité: 0.07
ID vidéo: cvZPq2GSvcM - Similarité: 0.08
ID vidéo: BOsV6spNbpQ - Similarité: 0.08
ID vidéo: 0ZhPQEdANGk - Similarité: 0.09
ID vidéo: mwpOSOWT7JM - Similarité: 0.10
ID vidéo: fpP4KN7bN5U - Similarité: 0.10
ID vidéo: Mjet7DYtYK4 - Similarité: 0.10
ID vidéo: kIuYrcF8fJ4 - Similarité: 0.11
ID vidéo: UVRDdw79eyo - Similarité: 0.11
ID vidéo: Npz3P9yABz8 - Similarité: 0.11
ID vidéo: eBEds_afLN0 - Similarité: 0.11
ID vidéo: 2bgtNHirdpg - Similarité: 0.12
ID vidéo: okbopJLO9as - Similarité: 0.12
ID vidéo: NLVL2074Og4 - Similarité: 0.13
ID vidéo: Jmyrz

In [26]:
for videos_sur_015 in videos_sur_015:
    print(f"ID vidéo: {videos_sur_015['id_video']} - Similarité: {videos_sur_015['similarity']:.2f}")

ID vidéo: IQDS8x3RoU0 - Similarité: 0.15
ID vidéo: z6OIoU88m6s - Similarité: 0.15
ID vidéo: Fpb2m32m3JU - Similarité: 0.16
ID vidéo: D3D9gO05SPI - Similarité: 0.16
ID vidéo: 6LEvKg8qKao - Similarité: 0.16
ID vidéo: mbWjjECz8dY - Similarité: 0.16
ID vidéo: i0IFbGzq-VQ - Similarité: 0.17
ID vidéo: 31dKCfWtJmI - Similarité: 0.17
ID vidéo: vmjxKZ4lzEk - Similarité: 0.17
ID vidéo: mde8rq8WG9w - Similarité: 0.17
ID vidéo: IpSDnSFd9-Y - Similarité: 0.17
ID vidéo: d06ssNOndOk - Similarité: 0.17
ID vidéo: js3S13MvH9M - Similarité: 0.17
ID vidéo: ZQ8hWTDNRoA - Similarité: 0.17
ID vidéo: nM6bhY-oGk8 - Similarité: 0.17
ID vidéo: q3cZfxeG4zQ - Similarité: 0.17
ID vidéo: nNzCrvIHJek - Similarité: 0.17
ID vidéo: rGePc1eJyUQ - Similarité: 0.17
ID vidéo: pPeKkd3PVh4 - Similarité: 0.17
ID vidéo: p2RsgaVauCM - Similarité: 0.17
ID vidéo: pWNzDB3PRFI - Similarité: 0.17
ID vidéo: 7q3d5BrinCM - Similarité: 0.17
ID vidéo: gkV3ywZiDLM - Similarité: 0.17
ID vidéo: LtYzN2Slcm0 - Similarité: 0.18
ID vidéo: GNkQq4

# With LLM

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
gemini_flash_1 = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=os.getenv("GOOGLE_API_KEY_1"))
gemini_flash_2 = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=os.getenv("GOOGLE_API_KEY_2"))

In [ ]:
sysprompt = """
# La definition d'autossuffinnce

L'autosuffisance est la démarche visant à acquérir la capacité de subvenir par soi-même à ses besoins fondamentaux, 
en premier lieu alimentaires par l'autoconsommation – c'est-à-dire produire, récolter et conserver un maximum de sa propre nourriture, 
souvent en privilégiant le bio, le local et le saisonnier. 

L'autosuffisance Plus qu'une simple recherche d'autonomie matérielle, elle représente un engagement pour être moins dépendant du système économique et social extérieur, impliquant des choix concrets comme trouver un lieu propice et le concevoir judicieusement (par exemple en permaculture), 
ainsi qu'un changement dans la manière de valoriser son temps et de consommer, formant ainsi le fondement d'une vie plus autonome.

---

# Votre mission

- Décider si une vidéo concerne la thématique de l'autosuffisance en se basant sur ses métadonnées (titre, description, tags).  
- Les vlogs, les interviews et les vidéos de partage d'expériences en lien avec l'autosuffisance sont intéressants.  
- Les tutoriels et les vidéos de conseils sont également pertinents.  
- Les vidéos présentant une technique liée à l'autosuffisance ou y contribuant sont utiles.  
- Attribuer un score entier de 1 à 10 pour évaluer la pertinence de la vidéo par rapport à la thématique de l'autosuffisance.

---

# Votre réponse  

- La réponse doit être au format JSON :  
  
    "decision": "oui ou non",
    "justification": "Justifiez votre décision avec des arguments"
    "score": votre èvaluation
 
"""

userprompt = """
Titre
---
{titre}
Description
---
{description}
tags
---
{tags}
"""

prompt = ChatPromptTemplate([
    ("system", sysprompt),
    ("user", userprompt)
])

chain_1 = prompt | gemini_flash_1
chain_2 = prompt | gemini_flash_2


- Test

In [ ]:
input = {"titre":"Tuteurer et redresser un arbre 🌳",
         "description":"Dans cette vidéo nous allons voir ensemble comment mettre un tuteur afin de redresser un arbre qui penche suite à la tempête.\nCette vidéo vous aideras pour placer un tuteur en travers , sangler un arbre selon le sens du vent,\n\nIl existe plusieurs facon de tuteurer un arbre, haubanage, tripode,.. mais dans le cas présent un tuteur en biais suffiras puisque l'arbre n'est pas trés gros.\n\n\n▬▬▬▬▬▬▬▬▬▬▬ POUR ALLER PLUS LOIN  ▬▬▬▬▬▬▬▬▬▬▬\n\n💡 Abonnez-vous pour me soutenir 🧡\n\n\n💡Rejoignez-nous sur twitter\net tik tok\n\n\n▬▬▬▬▬▬▬▬▬▬▬ CHAPITRES DE CETTE VIDÉO ▬▬▬▬▬▬▬▬▬▬\n\n00:00 : Introduction\n00:23 : Mise en place tuteur\n01:54 : Sanglage et Redresser l'arbre\n02:54 : Sens du vent et détails tuteurage\n\n▬▬▬▬▬▬▬▬▬▬▬ A PROPOS DE CETTE VIDÉO ▬▬▬▬▬▬▬▬▬▬▬\n\nLe Jardin du moulin\nFrancois\n#potager #jardindumoulin",
         "tags":', '.join( [
      "TCS",
      "Agriculture Biologique",
      "Bio",
      "GAB 85",
      "FNAB",
      "Vendée",
      "Techniques Culturales Simplifiées",
      "GAEC Les Jonquilles",
      "ECOPHYTO",
      "Demain Vendée",
      "Conservation des sols",
      "Sol vivant",
      "Productions végétales",
      "Maïs"
    ])}
print(input)
print(chain_2.invoke(input))

In [ ]:
def cleanAnswer(answer):
    answer = answer.strip("`")   # Supprimer tous les backticks
    if answer.startswith("json"):
        answer = answer[4:].strip() 
    return  answer

In [ ]:
videosF2 = openJson("../jsons/videosF2.json")
llmEvaluation = []
count1 = 0
count2 = 0
temp = 0

for video in tqdm(videosF2):
    input = {"titre":video['titre_video'],
             "description":video['description'],
             "tags":', '.join(video['tags']),}
    try:
        if count1  <= 13:
            print("KEY 1")
        
            answer = cleanAnswer(chain_1.invoke(input))
            #print(video['id_video'],'/n',answer)
            evaluation = {}
            evaluation['id_video']=video['id_video']
            evaluation.update(json.loads(answer))
            llmEvaluation.append(evaluation)
            
            count1+=1
            temp+=1            
             
            print("count1 ",count1)  
            
        if count1  > 13 and count2 <= 13:
            print("KEY 2")
            
            answer =  cleanAnswer(chain_2.invoke(input))
            #print(video['id_video'],'/n',answer)
            evaluation = {}
            evaluation['id_video']=video['id_video']
            evaluation.update(json.loads(answer))
            llmEvaluation.append(evaluation)
            
            count2+=1
            temp+=1
            print("count2 ",count2)  
            
        if count1 > 13 and count2 > 13 :
            print("sleep for 1 min")
            time.sleep(60)
            count1 = 0
            count2 = 0
        

        if temp >= 100:
            saveJson("../jsons/llmEvaluation.json",llmEvaluation)
            temp=0
            
    except Exception as e:
        print(f"error {e}")

## Confusion Matrix

- Seperate the videos classified as not relevent

In [ ]:
llmEvaluations = openJson("../jsons/llmEvaluation.json")
videosF2 = openJson("../jsons/videosF2.json")

def getVideo(ID):
    for video in videosF2:
        if video['id_video']==ID:
            return video

In [ ]:
len(videosF2)

In [ ]:
llmVideosNon = []
llmVideosOui = []

for evaluation in llmEvaluations:
    try:
        video = getVideo(evaluation['id_video'])
        video.update(evaluation)
        
        if evaluation['decision'] =='non':
            llmVideosNon.append(video)
        else :
            if len(llmVideosOui)
            llmVideosOui.append(video)
    except:
        print(evaluation)
        break
    
saveJson("../jsons1/llmVideosNon.json",llmVideosNon)
saveJson("../jsons1/llmVideosOui.json",llmVideosOui)

In [ ]:
llmVideosNon = openJson("../jsons1/llmVideosNon.json")
llmVideosOui = openJson("../jsons1/llmVideosOui.json")

print(len(llmVideosNon))
print(len(llmVideosOui))
print(len(llmVideosNon)+len(llmVideosOui))

- Validate the llmResults

In [ ]:
channelsF1 = openJson("../jsons/channelsF1.json")

def getChannelName(ID):
    for channel in channelsF1:
        if channel['id_chaine']==ID:
            return channel['nom_chaine'] 

In [ ]:
names = []
for videoOui in llmVideosOui:
    names.append(getChannelName(videoOui['id_chaine']))
    
namesCounter = Counter(names)    

In [ ]:
namesCounter

len(namesCounter)

In [ ]:
namesCounter

In [ ]:
scorellm = []
for v in videosNon:
    scorellm.append(v['score'])

In [ ]:
max(scorellm)

In [ ]:
import pandas as pd

data = []
for video in videosNon:
    data.append({
        "lien_video": f"https://www.youtube.com/watch?v={video['id_video']}",
        "titre": video["titre_video"],
        "decision": video["decision"],
        "justification": video["justification"],
        "score": video["score"]
    })

df = pd.DataFrame(data)
df.to_excel("../jsons/videosNon.xlsx", index=False)


- Number of search queries

In [ ]:
channelsR1 = openJson("../jsons/channelsR1.json")

In [ ]:
queriesCounter = []
for video in videosF2:
    for channel in channelsR1:
        if channel['id_chaine'] == video['id_chaine']:
            channelname = channel['nom_chaine']
    queriesCounter.append({
        'id_video': video['id_video'],
        'numQueries': len(video['requete']),
        'channelname': channelname
    })

queriesCounter = sorted(queriesCounter, key=lambda x: x['numQueries'], reverse=True)

total_queries = sum(item['numQueries'] for item in queriesCounter)
nombre_videos = len(queriesCounter)
moyenne_queries = total_queries / nombre_videos

print(queriesCounter[:5])
print(moyenne_queries)

In [ ]:
queriesCounterNon = []
for video in videosNon:
    for channel in channelsR1:
        if channel['id_chaine']==video['id_chaine']:
            channelname = channel['nom_chaine']
    queriesCounterNon.append(
        {
            'id_video':video['id_video'],
            'numQueries':len(video['requete']),
            'channelname':channelname
        }
    )

queriesCounterNon = sorted(queriesCounterNon, key=lambda x: x['numQueries'], reverse=True)

videos_less_than_2_queries = [video for video in queriesCounterNon if video['numQueries'] <= 2]
percentage = (len(videos_less_than_2_queries) / len(queriesCounterNon)) * 100
print(f"Pourcentage de vidéos (Parmi les 45 non-pertinents) avec <= 2 requêtes : {percentage:.2f}%")


## Indice de pertinence

I.P = llmScore × α + numQueries × β

- On teste au depart avec alpha= 0.7 et beta = 0.3

In [ ]:
llmEvaluations = openJson("../jsons/llmEvaluation.json")
videosF2 = openJson("../jsons/videosF2.json")

def getnbrQueries(ID):
    for video in videosF2:
        if video['id_video']==ID:
            return len(video['requete'])

In [ ]:
def calculateIP(alpha,beta):
    echantillonIP = []

    for eval in llmEvaluations:
        nbrQueries = getnbrQueries(eval['id_video'])
        llmScore = eval['score']
        ip = llmScore*alpha + nbrQueries*beta
        echantillonIP.append(
            {
                'id_video':eval['id_video'],
                'llmScore':eval['score'],
                'nbrQueries':nbrQueries,
                'IP':ip
            }
        )
    return echantillonIP    

In [ ]:
echantillonIP = calculateIP(0.7,0.3)
saveJson("../jsons/echantillonIP.json",echantillonIP)

In [ ]:
len(echantillonIP)

In [ ]:
import numpy as np

ip_values = [item['IP'] for item in echantillonIP]

plt.boxplot(ip_values, vert=True)
plt.title('Boxplot des valeurs de IP')
plt.show()

min_val = np.min(ip_values)
q1 = np.percentile(ip_values, 25)
median = np.median(ip_values)
q3 = np.percentile(ip_values, 75)
max_val = np.max(ip_values)

iqr = q3 - q1
lower_limit = q1 - 1.5 * iqr
upper_limit = q3 + 1.5 * iqr

print(f'Min     = {min_val:.2f}')
print(f'Q1      = {q1:.2f}')
print(f'Median  = {median:.2f}')
print(f'Q3      = {q3:.2f}')
print(f'Max     = {max_val:.2f}')
print(f'IQR     = {iqr:.2f}')
print(f'Limite inférieure des moustaches = {lower_limit:.2f}')
print(f'Limite supérieure des moustaches = {upper_limit:.2f}')


In [ ]:
echantillonIP=openJson("../jsons/echantillonIP.json")
filtredVidoesIP = []
for ech in echantillonIP:
    if ech['IP'] < 2.9:
        filtredVidoesIP.append(ech)
    
saveJson("../jsons/filtredVidoesIP.json",filtredVidoesIP)

In [ ]:
filtredVidoesIP = openJson("../jsons/filtredVidoesIP.json")
len(filtredVidoesIP)

In [ ]:
llmVideosNon = openJson("../jsons/llmVideosNon.json")
len(llmVideosNon)

In [ ]:
for filteredvid in filtredVidoesIP:
    exist = False
    for vid in llmVideosNon:
        if filteredvid['id_video']==vid['id_video']:
            exist = True
            break
    if exist == False:
        print(filteredvid['id_video'])    

In [ ]:
# RmN529MFKpU,8waoAttTffY doit revenir a LLM non
# QCcYN8qIOAI est pertinente 